In [8]:
import numpy as np
import pandas as pd
from scipy.stats import skew, kurtosis, entropy
from scipy.signal import find_peaks, welch, cwt, ricker
from scipy.spatial.distance import pdist, squareform
from statsmodels.tsa.stattools import acf
import os
import pywt

In [11]:

def extract_melodic_progression_features(mfcc_matrix):
    features = []
    names = []
    
    widths = np.arange(1, 20)
    wavelet = 'gaus1'
    for i in range(3):  # Analyze first 3 MFCCs
        cwt_matrix, _ = pywt.cwt(mfcc_matrix[i], widths, wavelet)
        
        features.extend([
            np.mean(np.abs(cwt_matrix)),  # Average pattern strength
            np.std(cwt_matrix),           # Pattern variability
            np.sum(np.abs(np.diff(cwt_matrix, axis=0))) / cwt_matrix.shape[1]  # Melodic change rate
        ])
        names.extend([f'melodic_strength_{i}', f'melodic_var_{i}', f'melodic_change_{i}'])
    
    # Tonal transition matrix (capturing melodic progressions)
    quantized_mfccs = np.round(mfcc_matrix[1:5] * 4) / 4  # Quantize to reduce noise
    transitions = np.zeros((16, 16))
    
    for i in range(quantized_mfccs.shape[1] - 1):
        current_state = np.sum(quantized_mfccs[:,i] * np.array([4, 3, 2, 1]))
        next_state = np.sum(quantized_mfccs[:,i+1] * np.array([4, 3, 2, 1]))
        transitions[int(current_state % 16), int(next_state % 16)] += 1
    
    # Extract features from transition matrix
    features.extend([
        np.count_nonzero(transitions) / transitions.size,  # Transition diversity
        entropy(transitions.flatten() + 1e-10),            # Transition entropy
        np.max(transitions) / np.mean(transitions)         # Dominant transition strength
    ])
    names.extend(['tonal_diversity', 'tonal_entropy', 'tonal_dominance'])
    
    return features, names

def extract_cultural_rhythm_features(mfcc_matrix):
    features = []
    names = []

    # 1. Taal/Meter Analysis using autocorrelation
    onset_env = np.sum(np.abs(np.diff(mfcc_matrix, axis=1)), axis=0)
    onset_ac = acf(onset_env, nlags=86)
    
    # Find peaks in autocorrelation to detect rhythm cycles
    peaks, properties = find_peaks(onset_ac, distance=10)
    if len(peaks) > 0:
        rhythm_features = [
            len(peaks),                    # Number of distinct rhythm cycles
            np.mean(onset_ac[peaks]),      # Average rhythm strength
            np.std(np.diff(peaks)),        # Rhythm regularity
            np.max(onset_ac[peaks]) / np.mean(onset_ac[peaks])  # Dominant rhythm strength
        ]
    else:
        rhythm_features = [0, 0, 0, 0]
    
    features.extend(rhythm_features)
    names.extend(['rhythm_cycles', 'rhythm_strength', 'rhythm_regularity', 'rhythm_dominance'])
    
    # 2. Complex rhythm pattern detection using time-warped segments
    segment_length = min(86, mfcc_matrix.shape[1] // 4)
    rhythm_patterns = []
    
    for i in range(4):
        segment = mfcc_matrix[:, i*segment_length:(i+1)*segment_length]
        rhythm_patterns.append(np.sum(np.abs(np.diff(segment, axis=1)), axis=0))
    
    # Calculate pattern similarity matrix
    pattern_sim = squareform(pdist(np.array(rhythm_patterns)))
    features.extend([
        np.mean(pattern_sim),     # Average pattern similarity
        np.min(pattern_sim[pattern_sim > 0]) if np.any(pattern_sim > 0) else 0,  # Most similar patterns
        np.max(pattern_sim)       # Most different patterns
    ])
    names.extend(['pattern_similarity', 'pattern_repetition', 'pattern_variation'])
    
    return features, names

def extract_vocal_ornament_features(mfcc_matrix):

    features = []
    names = []
    
    # 1. Short-time frequency analysis for ornament detection
    frame_size = 10
    hop_size = 5
    num_frames = (mfcc_matrix.shape[1] - frame_size) // hop_size
    
    ornament_stats = []
    for i in range(num_frames):
        frame = mfcc_matrix[:5, i*hop_size:i*hop_size+frame_size]
        
        # Detect rapid pitch changes (gamak)
        pitch_changes = np.sum(np.abs(np.diff(frame, axis=1)))
        
        # Detect smooth pitch slides (meend)
        pitch_smoothness = np.mean(np.abs(np.diff(np.diff(frame, axis=1), axis=1)))
        
        ornament_stats.append([pitch_changes, pitch_smoothness])
    
    ornament_stats = np.array(ornament_stats)
    
    # Calculate ornament features
    for i in range(2):
        features.extend([
            np.mean(ornament_stats[:, i]),  # Average ornament usage
            np.std(ornament_stats[:, i]),   # Ornament variability
            np.percentile(ornament_stats[:, i], 90),  # Strong ornament presence
            len(find_peaks(ornament_stats[:, i])[0]) / len(ornament_stats)  # Ornament density
        ])
    names.extend([
        'gamak_avg', 'gamak_var', 'gamak_strength', 'gamak_density',
        'meend_avg', 'meend_var', 'meend_strength', 'meend_density'
    ])
    
    # 2. Vibrato analysis using frequency modulation
    for i in range(3):
        coeff = mfcc_matrix[i]
        freqs, psd = welch(np.diff(coeff), fs=86)
        
        # Analyze different vibrato types (4-8 Hz for classical, 5-7 Hz for western)
        classical_vibrato = np.sum(psd[(freqs >= 4) & (freqs <= 8)])
        western_vibrato = np.sum(psd[(freqs >= 5) & (freqs <= 7)])
        
        features.extend([classical_vibrato, western_vibrato])
        names.extend([f'classical_vibrato_{i}', f'western_vibrato_{i}'])
    
    return features, names

def extract_production_style_features(mfcc_matrix):

    features = []
    names = []
    
    # 1. Spectral flatness across different frequency bands
    bands = [(1,4), (5,8), (9,12), (13,16)]
    for start, end in bands:
        band = mfcc_matrix[start:end]
        flatness = np.exp(np.mean(np.log(np.abs(band) + 1e-10))) / (np.mean(np.abs(band)) + 1e-10)
        features.append(flatness)
    names.extend([f'spectral_flatness_band_{i}' for i in range(len(bands))])
    
    # 2. Production quality metrics
    spectral_flux = np.mean(np.abs(np.diff(mfcc_matrix, axis=1)), axis=1)
    features.extend([
        np.mean(spectral_flux),    # Overall dynamic range
        np.std(spectral_flux),     # Dynamic variation
        skew(spectral_flux),       # Dynamic asymmetry
        kurtosis(spectral_flux)    # Dynamic peakedness
    ])
    names.extend(['dynamic_range', 'dynamic_var', 'dynamic_skew', 'dynamic_kurt'])
    
    # 3. Microphone and recording quality indicators
    noise_floor = np.percentile(np.abs(mfcc_matrix), 10, axis=1)
    features.extend([
        np.mean(noise_floor),      # Average noise level
        np.std(noise_floor),       # Noise consistency
        np.max(noise_floor)        # Peak noise level
    ])
    names.extend(['noise_level', 'noise_consistency', 'peak_noise'])
    
    return features, names

def extract_ensemble_features(mfcc_matrix):

    features = []
    names = []
    
    # 1. Spectral density analysis
    freq_bands = [(1,4), (5,8), (9,12), (13,16), (17,19)]
    density_features = []
    
    for start, end in freq_bands:
        band = mfcc_matrix[start:end]
        # Calculate spectral density features
        density = np.sum(np.abs(band) > np.mean(np.abs(band))) / len(band)
        density_features.append(density)
    
    features.extend(density_features)
    names.extend([f'freq_band_density_{i}' for i in range(len(freq_bands))])
    
    return features, names

def extract_national_anthem_features(mfcc_matrix):

    features = []
    names = []
    
    # 1. Tempo stability analysis
    onset_env = np.sum(np.abs(np.diff(mfcc_matrix, axis=1)), axis=0)
    peaks, _ = find_peaks(onset_env, distance=20)
    if len(peaks) > 1:
        intervals = np.diff(peaks)
        tempo_stability = np.std(intervals) / np.mean(intervals)
        features.append(tempo_stability)
    else:
        features.append(0)
    names.append('tempo_stability')
    
    # 2. Pronunciation clarity (using MFCC transitions)
    transitions = np.diff(mfcc_matrix, axis=1)
    clarity_score = np.mean(np.abs(transitions)) / np.std(transitions)
    features.append(clarity_score)
    names.append('pronunciation_clarity')
    
    # 3. Rhythmic pattern matching (specific to anthem)
    segment_length = min(86, mfcc_matrix.shape[1] // 4)
    pattern_scores = []
    for i in range(4):
        segment = mfcc_matrix[:, i*segment_length:(i+1)*segment_length]
        pattern_scores.append(np.std(np.mean(segment, axis=0)))
    features.extend(pattern_scores)
    names.extend([f'rhythm_pattern_{i}' for i in range(4)])
    
    return features, names

def extract_bhav_geet_features(mfcc_matrix):

    features = []
    names = []
    
    # 1. Tempo analysis for slower patterns
    tempo_envelope = np.sum(np.abs(np.diff(mfcc_matrix, axis=1)), axis=0)
    peaks, _ = find_peaks(tempo_envelope, distance=30)  # Larger distance for slower tempo
    tempo_density = len(peaks) / mfcc_matrix.shape[1]
    features.append(tempo_density)
    names.append('tempo_density')
    
    # 2. Emotional modulation detection
    modulation = np.std(mfcc_matrix, axis=1)
    emotional_range = np.max(modulation) / np.mean(modulation)
    features.append(emotional_range)
    names.append('emotional_range')
    
    # 3. Percussion absence feature
    percussion_band = np.mean(mfcc_matrix[1:4, :], axis=0)  # Lower frequency bands
    percussion_intensity = np.percentile(percussion_band, 90)
    features.append(percussion_intensity)
    names.append('percussion_intensity')
    
    return features, names

def extract_lavni_features(mfcc_matrix):

    features = []
    names = []
    
    # 1. Fast tempo detection
    tempo_env = np.sum(np.abs(np.diff(mfcc_matrix, axis=1)), axis=0)
    peaks, _ = find_peaks(tempo_env, distance=10)  # Smaller distance for faster tempo
    if len(peaks) > 1:
        peak_intervals = np.diff(peaks)
        features.extend([
            np.mean(peak_intervals),  # Average beat interval
            np.percentile(peak_intervals, 25),  # Fast tempo indicator
            len(peaks) / mfcc_matrix.shape[1]  # Beat density
        ])
    else:
        features.extend([0, 0, 0])
    names.extend(['beat_interval', 'fast_tempo_indicator', 'beat_density'])
    
    # 2. Rhythmic pattern strength
    rhythm_strength = np.std(np.mean(mfcc_matrix[1:5, :], axis=0))
    features.append(rhythm_strength)
    names.append('rhythm_strength')
    
    # 3. Percussion energy
    percussion_band = np.mean(mfcc_matrix[1:4, :], axis=0)
    features.append(np.max(percussion_band))
    names.append('percussion_energy')
    
    return features, names

def extract_singer_specific_features(mfcc_matrix):

    features = []
    names = []
    
    # 1. Vocal texture analysis
    for i in range(1, 6):  # Analyze first 5 MFCCs
        coeff = mfcc_matrix[i]
        features.extend([
            np.mean(coeff),
            np.std(coeff),
            skew(coeff),
            kurtosis(coeff)
        ])
        names.extend([
            f'vocal_texture_mean_{i}',
            f'vocal_texture_std_{i}',
            f'vocal_texture_skew_{i}',
            f'vocal_texture_kurt_{i}'
        ])
    
    # 2. Pitch range analysis
    pitch_range = np.ptp(mfcc_matrix[1:5, :], axis=1)  # Peak to peak of pitch-related coefficients
    features.extend(pitch_range)
    names.extend([f'pitch_range_{i}' for i in range(len(pitch_range))])
    
    # 3. Vibrato analysis
    for i in range(1, 4):
        # Analyze frequency content of MFCC variations
        freqs, psd = welch(np.diff(mfcc_matrix[i]), fs=86)  # 86 frames per second
        vibrato_power = np.sum(psd[(freqs >= 4) & (freqs <= 8)])  # 4-8 Hz vibrato range
        features.append(vibrato_power)
        names.append(f'vibrato_power_{i}')
    
    # 4. Style signatures
    # Transition patterns between notes
    transitions = np.diff(mfcc_matrix[1:5, :], axis=1)
    transition_stats = [
        np.mean(np.abs(transitions)),  # Average transition speed
        np.std(transitions),           # Transition variability
        skew(transitions.flatten()),   # Transition asymmetry
        kurtosis(transitions.flatten())# Transition peakedness
    ]
    features.extend(transition_stats)
    names.extend(['transition_speed', 'transition_var', 'transition_skew', 'transition_kurt'])
    
    return features, names

def extract_western_music_features(mfcc_matrix):

    features = []
    names = []
    
    # 1. Scale system analysis
    scale_features = np.mean(mfcc_matrix[2:8, :], axis=1)  # Focus on mid-range coefficients
    features.extend([
        np.mean(scale_features),
        np.std(scale_features),
        skew(scale_features),
        kurtosis(scale_features)
    ])
    names.extend(['scale_mean', 'scale_std', 'scale_skew', 'scale_kurt'])
    
    # 2. Production style features
    # Analyze spectral flatness and contrast
    spectral_contrast = np.max(mfcc_matrix, axis=0) - np.min(mfcc_matrix, axis=0)
    features.extend([
        np.mean(spectral_contrast),
        np.std(spectral_contrast),
        np.percentile(spectral_contrast, 90)
    ])
    names.extend(['production_contrast_mean', 'production_contrast_std', 'production_contrast_peak'])
    
    # 3. Instrument mix analysis
    # Analyze different frequency bands
    bands = [
        (1, 4),   # Low frequency
        (5, 9),   # Mid frequency
        (10, 15), # High frequency
        (16, 19)  # Very high frequency
    ]
    
    for i, (start, end) in enumerate(bands):
        band = mfcc_matrix[start:end, :]
        band_stats = [
            np.mean(np.std(band, axis=1)),  # Variation within band
            np.mean(np.abs(np.diff(band, axis=1)))  # Temporal changes
        ]
        features.extend(band_stats)
        names.extend([f'band_{i}_variation', f'band_{i}_temporal'])
    
    return features, names

In [ ]:
def extract_features_from_csv(csv_file):
    # Load the MFCC data from the CSV file
    data = pd.read_csv(csv_file)
    # Assuming that the MFCC matrix is stored in columns 0 to 19
    mfcc_matrix = data.iloc[:, :20].values  # Transpose to match the expected shape (MFCC x Time)
    
    # Extract features using the defined functions
    all_features = []
    all_names = []
    
    # Call the feature extraction functions
    features, names = extract_melodic_progression_features(mfcc_matrix)
    all_features.extend(features)
    all_names.extend(names)
    
    features, names = extract_cultural_rhythm_features(mfcc_matrix)
    all_features.extend(features)
    all_names.extend(names)
    
    features, names = extract_vocal_ornament_features(mfcc_matrix)
    all_features.extend(features)
    all_names.extend(names)
    
    features, names = extract_production_style_features(mfcc_matrix)
    all_features.extend(features)
    all_names.extend(names)
    
    features, names = extract_ensemble_features(mfcc_matrix)
    all_features.extend(features)
    all_names.extend(names)
    
    # Additional custom feature extraction for different types
    features, names = extract_national_anthem_features(mfcc_matrix)
    all_features.extend(features)
    all_names.extend(names)
    
    features, names = extract_bhav_geet_features(mfcc_matrix)
    all_features.extend(features)
    all_names.extend(names)
    
    features, names = extract_lavni_features(mfcc_matrix)
    all_features.extend(features)
    all_names.extend(names)
    
    features, names = extract_singer_specific_features(mfcc_matrix)
    all_features.extend(features)
    all_names.extend(names)
    
    features, names = extract_western_music_features(mfcc_matrix)
    all_features.extend(features)
    all_names.extend(names)
    
    return all_features, all_names

def process_all_files_in_folder(folder_path, output_csv):
    all_data = []
    
    for filename in os.listdir(folder_path):
        if filename.endswith(".csv"):
            file_path = os.path.join(folder_path, filename)
            print(f"Processing file: {filename}")
            
            # Extract features for the current file
            features, feature_names = extract_features_from_csv(file_path)
            
            # Create a row for the current file's features
            row = features
            row.append(filename)  # Add filename as an identifier for the file
            
            all_data.append(row)
    
    # Convert the list of all data to a DataFrame
    feature_df = pd.DataFrame(all_data, columns=feature_names + ['filename'])
    
    # Save the DataFrame to the output CSV
    feature_df.to_csv(output_csv, index=False)
    print(f"All features saved to {output_csv}")

# Example usage
folder_path = 'MFCC-files/'
output_csv = 'MFCC-files-features.csv'
process_all_files_in_folder(folder_path, output_csv)

Processing file: 01-MFCC.csv


d:\MY FILES\IITB\Sem 3\DS203\env\Lib\site-packages\scipy\signal\_spectral_py.py:600: UserWarning: nperseg = 256 is greater than input length  = 19, using nperseg = 19
  freqs, _, Pxy = _spectral_helper(x, y, fs, window, nperseg, noverlap,
d:\MY FILES\IITB\Sem 3\DS203\env\Lib\site-packages\numpy\_core\_methods.py:218: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
d:\MY FILES\IITB\Sem 3\DS203\env\Lib\site-packages\numpy\_core\_methods.py:175: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
d:\MY FILES\IITB\Sem 3\DS203\env\Lib\site-packages\numpy\_core\_methods.py:210: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
d:\MY FILES\IITB\Sem 3\DS203\env\Lib\site-packages\scipy\signal\_spectral_py.py:600: UserWarning: nperseg = 256 is greater than input length  = 19, using nperseg = 19
  freqs, _, Pxy = _spectral_helper(x, y, f

Processing file: 02-MFCC.csv
Processing file: 03-MFCC.csv


d:\MY FILES\IITB\Sem 3\DS203\env\Lib\site-packages\numpy\_core\_methods.py:218: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
d:\MY FILES\IITB\Sem 3\DS203\env\Lib\site-packages\numpy\_core\_methods.py:175: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
d:\MY FILES\IITB\Sem 3\DS203\env\Lib\site-packages\numpy\_core\_methods.py:210: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
d:\MY FILES\IITB\Sem 3\DS203\env\Lib\site-packages\scipy\signal\_spectral_py.py:600: UserWarning: nperseg = 256 is greater than input length  = 19, using nperseg = 19
  freqs, _, Pxy = _spectral_helper(x, y, fs, window, nperseg, noverlap,


Processing file: 04-MFCC.csv


...

d:\MY FILES\IITB\Sem 3\DS203\env\Lib\site-packages\scipy\signal\_spectral_py.py:600: UserWarning: nperseg = 256 is greater than input length  = 19, using nperseg = 19
  freqs, _, Pxy = _spectral_helper(x, y, fs, window, nperseg, noverlap,


Processing file: 98-MFCC.csv


d:\MY FILES\IITB\Sem 3\DS203\env\Lib\site-packages\numpy\_core\_methods.py:218: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
d:\MY FILES\IITB\Sem 3\DS203\env\Lib\site-packages\numpy\_core\_methods.py:175: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
d:\MY FILES\IITB\Sem 3\DS203\env\Lib\site-packages\numpy\_core\_methods.py:210: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
d:\MY FILES\IITB\Sem 3\DS203\env\Lib\site-packages\scipy\signal\_spectral_py.py:600: UserWarning: nperseg = 256 is greater than input length  = 19, using nperseg = 19
  freqs, _, Pxy = _spectral_helper(x, y, fs, window, nperseg, noverlap,


Processing file: 99-MFCC.csv
All features saved to MFCC-files-features.csv


d:\MY FILES\IITB\Sem 3\DS203\env\Lib\site-packages\statsmodels\tsa\stattools.py:693: RuntimeWarning: invalid value encountered in divide
  acf = avf[: nlags + 1] / avf[0]
d:\MY FILES\IITB\Sem 3\DS203\env\Lib\site-packages\scipy\signal\_spectral_py.py:600: UserWarning: nperseg = 256 is greater than input length  = 19, using nperseg = 19
  freqs, _, Pxy = _spectral_helper(x, y, fs, window, nperseg, noverlap,
C:\Users\Ronak\AppData\Local\Temp\ipykernel_32476\376018993.py:201: RuntimeWarning: invalid value encountered in scalar divide
  clarity_score = np.mean(np.abs(transitions)) / np.std(transitions)
C:\Users\Ronak\AppData\Local\Temp\ipykernel_32476\376018993.py:230: RuntimeWarning: invalid value encountered in scalar divide
  emotional_range = np.max(modulation) / np.mean(modulation)
